# PaperPi Structure
Supervisor will loop and poll each plugin for an update. Plugins are all`Plugin()` objects with a `poll` method

When polled `Plugin()` objects respond with a structured list (tuple/dict) with the following information:
* Priority(`int`): 0 (high) -- 10+ (low); plugins with negative values are excluded
* Update Required(`bool`): True update; False do not update

Plugin objects require the following parameters:
* configuration(`dict`) argparse/config.ini style dict
* resolution (this may be pulled from the configuration?)
* name(`str`) human readable name for logging and reference
* update_function(`func`): function that will run and provide updates and status

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
from itertools import cycle
from inspect import getfullargspec
from importlib import import_module



In [3]:
from library.CacheFiles import CacheFiles
from library.Plugin import Plugin

In [4]:
logging.basicConfig(level=logging.INFO)


In [5]:
logging.root.setLevel('DEBUG')

In [6]:
config = {
    'main': {
        'resolution': (500, 400)
    },
    'Plugin: Clock': {
        'layout': 'layout',
        'plugin': 'basic_clock',
#         'update_function': clock_plugin,
        'refresh_rate': 30,
        'min_display_time': 10,
        'max_priority': 1
    },
    'Plugin: LMS MacPlay': {
        'layout': 'twoColumn',
        'plugin': 'lms_client',
#         'update_function': slim_plugin,
        'refresh_rate': 5,
        'player_name': 'MacPlay',
        'min_display_time': 15,
        'max_priority': 0,
        'idle_timeout': 30
    },
#     'Plugin: Bogus': {
#         'layout': bogus_layout,
#         'update_function': bogus_plugin,
#         'refresh_rate': 5,
#         'min_display_time': 20,
#         'max_priority': -1,
#     },
    'Plugin: LibreSpot': {
        'layout': 'twoColumn',
        'plugin': 'librespot_client',
#         'update_function': 'librespot_client',
        'refresh_rate': 10,
        'player_name': 'SpoCon-Spotify',
        'idle_timeout': 15,
        'max_priority': 0,
    }
}


# Fonts in layouts are børked 
Fix all the paths:
- [x] move all the fonts to fonts directory at top level
- [ ] get the real path to each layout file:

        import os
        dir_path = os.path.dirname(os.path.realpath(__file__))
        
        'font': dir_path+'/../../fonts/Kanit/Kanit-Medium.ttf',
        
- [ ] test all the plugins

In [10]:
logging.debug('foo')

DEBUG:root:foo


In [12]:


logging.debug('FOO '*7)

cache = CacheFiles()
spec_kwargs = getfullargspec(Plugin).args

plugins = []

for section, values in config.items():
    if section.startswith('Plugin:'):
        logging.debug(f'Creating Plugin: [[ {section} ]]')

        my_config = {}
        # add all the spec_kwargs from the config
        plugin_kwargs = {}
        for key, val in values.items():
            if key in spec_kwargs:
                my_config[key] = val
            else:
                plugin_kwargs[key] = val
        
        my_config['name'] = section
        my_config['resolution'] = config['main']['resolution']
        my_config['config'] = plugin_kwargs
        my_config['cache'] = cache
        module = import_module(f'plugins.{values["plugin"]}')
        my_config['update_function'] = module.update_function
        my_config['layout'] = getattr(module.layout, values['layout'])
        my_plugin = Plugin(**my_config)
        plugins.append(my_plugin)

# for section, values in config.items():
#     my_config = {}
#     if section.startswith('Plugin:'):
# #         print(section)
# #         print(values)
#         # pass the entire config to Plugin() -- Plugin accepts **kwargs and will ignore unknowns
#         my_config = config[section] # add the entire config dictionary
# #         my_config['name'] = section # 
# #         my_config['resolution'] = config['main']['resolution']
# #         my_config['config'] = config[section]
# #         my_config['cache'] = cache # make a cache object available to all plugins (use the same one!)
# #         my_config['update_function'] = import_module('plugins.'+values['plugin']).update_function
# #         layout = import_module('plugins.'+values['plugin']+'.layout')
# #         print(values['layout'])
# #         my_config['layout'] = getattr(layout, values['layout'])
# #         # create the object 
# #         my_plugin = Plugin(**my_config)
        
# #         plugins.append(my_plugin)
#         plugins.append(my_config)

# print(plugins)
        
        


DEBUG:root:FOO FOO FOO FOO FOO FOO FOO 
DEBUG:root:Creating Plugin: [[ Plugin: Clock ]]
DEBUG:root:creating layout
DEBUG:root:calculating layout for resolution (500, 400)
DEBUG:root:layout id(2952335104)
DEBUG:root:*****digit_time*****
DEBUG:root:checking layout keys
DEBUG:root:adding key: padding: 0
DEBUG:root:adding key: hcenter: False
DEBUG:root:adding key: vcenter: False
DEBUG:root:adding key: inverse: False
DEBUG:root:adding key: relative: False
DEBUG:root:adding key: font_size: None
DEBUG:root:adding key: maxchar: None
DEBUG:root:adding key: dimensions: None
DEBUG:root:adding key: scale_x: None
DEBUG:root:adding key: scale_y: None
DEBUG:root:dimensions: (500, 400)
DEBUG:root:section has absolute coordinates
DEBUG:root:coordinates: (0, 0)
DEBUG:root:calculating maximum font size for area: (500, 400)
DEBUG:root:using font: /home/pi/src/epd_display/paperpi/fonts/Kanit/Kanit-Medium.ttf
DEBUG:root:target X font dimension 588.2352941176471
DEBUG:root:target Y font dimension 150.0
DEBUG

OSError: cannot open resource

In [ ]:
from plugins import lms_client

In [ ]:
lms_client.layout.twoColumn

In [ ]:
from IPython.display import display


plugins = []

for section in config:
    if section.startswith('Plugin: '):
        my_config = config[section]
        my_config['name'] = section
        my_config['resolution'] = config['main']['resolution']
        my_config['config'] = config[section]
        my_config['cache'] = cache # make a cache object available to all plugins (use the same one!)
        my_plugin = Plugin(**my_config)
        my_plugin.update()
        my_dict = {'plugin': my_plugin,
#                    'display_timer': Update(),
#                    'hash': my_plugin.hash,
                   'min_display_time': my_plugin.min_display_time}
        
        plugins.append(my_dict)




plugin_cycle = cycle(plugins)
this_plugin = next(plugin_cycle)
this_plugin_timer = Update()
max_priority = -1
this_hash = ''

while True:
    logging.info('updating plugins')
    logging.getLogger().setLevel('INFO')
    priority_list = []
    for plugin in plugins:
        logging.debug(f'updating plugin: [[ {plugin["plugin"].name} ]]')
        plugin['plugin'].update()
        # record the priority of all actiuve plugins (priority >= 0)
        if plugin['plugin'].priority >= 0:
            priority_list.append(plugin['plugin'].priority)
        
    # priority increases as it approaches 0; negative priorities are considered inactive
    max_priority = min(priority_list)
    logging.info(f'maximum priority is set to: {max_priority}')
        

    logging.getLogger().setLevel('DEBUG')
    logging.info(f'plugin currently displayed << {this_plugin["plugin"].name} >>')
    if this_plugin_timer.last_updated > this_plugin['plugin'].min_display_time:
        logging.info(f'display timer expired; switching plugin')
        plugin_is_active = False
        
        # find the next active plugin in the cycle
        while not plugin_is_active:
            this_plugin = next(plugin_cycle)
            logging.debug(f'checking display priority of {this_plugin["plugin"].name}')
            logging.debug(f'max_priority: {max_priority}, this plugin priority {this_plugin["plugin"].priority}')
            # beef this up to check priority overall
            if this_plugin['plugin'].priority >= 0 and this_plugin['plugin'].priority <= max_priority:
                plugin_is_active = True
            else:
                plugin_is_active = False
                logging.debug(f'{this_plugin["plugin"].name} has a priority of {this_plugin["plugin"].priority}; skipping')
        
        logging.debug(f'using plugin {this_plugin["plugin"].name}')
        
        # if the hashes do not match, display the new image, update the hash
        if this_hash != this_plugin['plugin'].hash:
            logging.debug('plugin image has refreshed -- refreshing screen')
            this_hash = this_plugin['plugin'].hash
            display(this_plugin['plugin'].image)
        else:
            logging.debug('plugin image has not refreshed -- skipping screen refresh')
            # record the number of times the screen is not refreshed here
            # if that number grows past N, wipe the display and then display the image
            pass
        this_plugin_timer.update()
        
    time.sleep(1)
        
        
        
        
        
